# Is reCAPTCHAv2 Safe? - TIMM Implementation

**Educational Research Project - UFABC Artificial Intelligence Course**

This notebook implements a deep learning classifier using **PyTorch Image Models (timm)** to analyze the viability of reCAPTCHAv2 as a CAPTCHA method.

## Model Options
You can experiment with different pre-trained models by changing `model_name` in CONFIG:
- `efficientnet_b0` - Efficient and fast (default)
- `resnet50` - Classic architecture
- `vit_small_patch16_224` - Vision Transformer
- `convnext_tiny` - Modern ConvNet
- `mobilenetv3_large_100` - Lightweight mobile model

## Features
- Transfer learning with timm pre-trained models
- Data augmentation for improved generalization
- Early stopping to prevent overfitting
- Learning rate scheduling (Cosine Annealing)
- MPS/CUDA/CPU support
- Training visualization and metrics

In [ ]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import torch
import timm

In [ ]:
dataset_path = "../datasets"
shared_images_dir = Path("../dataset_shared/images")

max_images = 3000
N_FOLDS = 5

timm_main_models = [
    # --- CNNs ---
    "resnet50",                 # ResNet family
    "resnext50_32x4d",          # ResNeXt
    "seresnet50",               # SENet family
    "densenet121",              # DenseNet
    "efficientnet_b0",          # EfficientNet
    "mobilenetv3_large_100",    # MobileNet
    "convnext_base",            # ConvNeXt
    "regnety_008",              # RegNet
    "dpn92",                    # DPN (Dual Path)
    "skresnet50",               # Selective Kernel conv
    "selecsls60",               # Selective SLS
    "inception_v3",             # Inception
    "squeezenet1_1",            # SqueezeNet

    # --- Vision Transformers ---
    "vit_base_patch16_224",     # ViT
    "deit_base_patch16_224",    # DeiT
    "swin_base_patch4_window7_224",  # Swin Transformer
    "beit_base_patch16_224",    # BEiT
    "beitv2_base_patch16_224",  # BEiT v2
    "cait_s24_224",             # CaiT
    "tnt_s_patch16_224",        # TNT
    "coat_lite_small",          # CoaT
    "twins_svt_small",          # Twins-SVT
    "pvt_small",                # Pyramid Vision Transformer (PVT)
    "crossvit_small_240",       # CrossViT
    "volo_d1_224",              # VOLO

    # --- Hybrid CNN + Transformer ---
    "coatnet_0",                # CoAtNet
    "efficientformerv2_s1",     # EfficientFormerV2
    "nfnet_f0",                 # NFNet (large CNN hybrid-like)

    # --- Lightweight / Mobile ---
    "tinynet_a",                # TinyNet
    "mnasnet_100",              # MnasNet
    "hardcorenas_a",            # HardCoReNAS

    # --- Large / High-Performance ---
    "eva_giant_patch14_336",    # EVA (gigantic ViT model)
    "maxvit_base_tf_224",       # MaxViT
]

model_name = 'efficientnet_b0'
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

CONFIG = {
    'model_name': model_name,
    'img_size': 224,
    'batch_size': 32,
    'epochs': 2,
    'patience': 50,
    'lr': 0.001,
    'device': 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu',
    'num_workers': 4,
    'n_folds': N_FOLDS,  # ADICIONAR
    'save_dir': f'is_recaptchav2_safe/timm_experiment/{model_name}'
}

In [ ]:
from utils.dataset_download import DatasetDownloader
from utils.load_datasets import load_datasets

dataset_downloader = DatasetDownloader(dataset_path)
dataset_downloader.download_all()

df = load_datasets(dataset_path)
display(df.head())
display(df['From'].value_counts())
display(f"Total images loaded: {len(df)}")

In [ ]:
labels = df['Label'].unique()

label_count = {label: 0 for label in labels}

updt_df_dict = {
    "Image": [],
    "Filename": [],
    "Label": [],
    "From": []
}

# Create shared images directory once (outside the loop)
shared_images_dir.mkdir(parents=True, exist_ok=True)

# Create symlinks to all images once
print("Creating shared images directory...")
for idx, row in df.iterrows():
    if label_count[row['Label']] < max_images:
        src = Path(row['Image']).absolute()
        unique_filename = f"{row['From']}_{row['Filename']}"
        dst = shared_images_dir / unique_filename
        
        if not dst.exists():
            os.symlink(src, dst)
        label_count[row['Label']] += 1

        updt_df_dict["Image"].append(row["Image"])
        updt_df_dict["Filename"].append(row["Filename"])
        updt_df_dict["Label"].append(row["Label"])
        updt_df_dict["From"].append(row["From"])

print(f"Shared images directory created with {len(list(shared_images_dir.iterdir()))} images")

df = pd.DataFrame(updt_df_dict)
df.to_parquet(dataset_path / "datasets_reduced.parquet", index=False)

In [ ]:
try:
    display(df['Label'].value_counts())
except NameError:
    df = pd.read_parquet(os.path.join(dataset_path, 'datasets_reduced.parquet'), engine='pyarrow')
    display(df['Label'].value_counts())

In [ ]:
import shutil

# Setup k-fold
labels = df['Label'].unique()

# Store results from all folds
all_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['Label'])):
    print(f"\n{'='*50}")
    print(f"FOLD {fold + 1}/{N_FOLDS}")
    print(f"{'='*50}")
    
    # Split data for this fold
    train_df = df.iloc[train_idx].reset_index(drop=True)
    val_df = df.iloc[val_idx].reset_index(drop=True)
    
    print(f"Train: {len(train_df)}, Val: {len(val_df)}")
    print(f"Train label distribution:\n{train_df['Label'].value_counts()}")
    print(f"Val label distribution:\n{val_df['Label'].value_counts()}")
    
    # Create fold-specific dataset structure
    dataset_root = Path(f"../dataset_fold{fold}")
    labels_file = dataset_root / "labels.txt"
    
    dataset_root.mkdir(parents=True, exist_ok=True)
    
    # Build list of images with their splits for this fold
    all_images = []
    
    # Add training images
    for idx, row in train_df.iterrows():
        unique_filename = f"{row['From']}_{row['Filename']}"
        all_images.append({
            'filename': unique_filename,
            'label': row['Label'],
            'split': 'train'
        })
    
    # Add validation images
    for idx, row in val_df.iterrows():
        unique_filename = f"{row['From']}_{row['Filename']}"
        all_images.append({
            'filename': unique_filename,
            'label': row['Label'],
            'split': 'val'
        })
    
    # Create labels file with format: filename label split
    with open(labels_file, 'w') as f:
        f.write("filename\tlabel\tsplit\n")
        for img_info in all_images:
            f.write(f"{img_info['filename']}\t{img_info['label']}\t{img_info['split']}\n")
    
    # For TIMM classification, create train/val split directories
    # with symlinks pointing to the shared images folder
    train_dir = dataset_root / "train"
    val_dir = dataset_root / "val"
    
    # Remove old train/val if they exist
    if train_dir.exists():
        shutil.rmtree(train_dir)
    if val_dir.exists():
        shutil.rmtree(val_dir)
    
    # Create label subdirectories
    for label in labels:
        (train_dir / label).mkdir(parents=True, exist_ok=True)
        (val_dir / label).mkdir(parents=True, exist_ok=True)
    
    # Create symlinks in train/val pointing to shared images folder
    for img_info in all_images:
        src = (shared_images_dir / img_info['filename']).absolute()
        if img_info['split'] == 'train':
            dst = train_dir / img_info['label'] / img_info['filename']
        else:
            dst = val_dir / img_info['label'] / img_info['filename']
        
        os.symlink(src, dst)
    
    print(f"Fold {fold + 1} dataset created!")
    print(f"  - Labels file: {labels_file}")

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
from timm.loss import LabelSmoothingCrossEntropy
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Configuration

print(f"Using device: {CONFIG['device']}")
print(f"Model: {CONFIG['model_name']}")

# Create save directory
Path(CONFIG['save_dir']).mkdir(parents=True, exist_ok=True)

In [ ]:
# Store results from all folds
all_fold_results = []


# Training and validation functions (definir FORA do loop)
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc='Training')
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        pbar.set_postfix({
            'loss': f'{running_loss / len(pbar):.4f}',
            'acc': f'{100. * correct / total:.2f}%'
        })

    return running_loss / len(train_loader), 100. * correct / total


def validate(model, val_loader, criterion, device):
    all_preds = []
    all_labels = []

    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        pbar = tqdm(val_loader, desc='Validation')
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            pbar.set_postfix({
                'loss': f'{running_loss / len(pbar):.4f}',
                'acc': f'{100. * correct / total:.2f}%'
            })

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    return running_loss / len(val_loader), 100. * correct / total, cm


# INÍCIO DO LOOP DE FOLDS
for fold in range(CONFIG['n_folds']):
    print(f"\n{'=' * 60}")
    print(f"PROCESSING FOLD {fold + 1}/{CONFIG['n_folds']}")
    print(f"{'=' * 60}")

    # Update data directory for current fold
    fold_data_dir = f'../dataset_fold{fold}'
    fold_save_dir = os.path.join(CONFIG['save_dir'], f'fold{fold}')
    Path(fold_save_dir).mkdir(parents=True, exist_ok=True)

    # Data Transforms
    train_transform = transforms.Compose([
        transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((CONFIG['img_size'], CONFIG['img_size'])),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load datasets for this fold
    train_dataset = datasets.ImageFolder(
        root=os.path.join(fold_data_dir, 'train'),
        transform=train_transform
    )

    val_dataset = datasets.ImageFolder(
        root=os.path.join(fold_data_dir, 'val'),
        transform=val_transform
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=True,
        num_workers=CONFIG['num_workers'],
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=False,
        num_workers=CONFIG['num_workers'],
        pin_memory=True
    )

    num_classes = len(train_dataset.classes)
    class_names = train_dataset.classes

    print(f"\nDataset Statistics:")
    print(f"Number of classes: {num_classes}")
    print(f"Classes: {class_names}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")

    # Create model using timm (NOVO modelo para cada fold)
    model = timm.create_model(
        CONFIG['model_name'],
        pretrained=True,
        num_classes=num_classes
    )

    model = model.to(CONFIG['device'])

    # Loss function and optimizer
    criterion = LabelSmoothingCrossEntropy(smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=0.01)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=10,
        T_mult=2,
        eta_min=1e-6
    )

    print(f"\nModel created: {CONFIG['model_name']}")
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

    # Training loop with early stopping
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'lr': []
    }

    best_val_acc = 0.0
    patience_counter = 0
    best_model_path = os.path.join(fold_save_dir, 'best.pt')
    last_model_path = os.path.join(fold_save_dir, 'last.pt')

    print("\n" + "=" * 50)
    print(f"Starting Training - Fold {fold + 1}")
    print("=" * 50)

    for epoch in range(CONFIG['epochs']):
        print(f"\nEpoch {epoch + 1}/{CONFIG['epochs']}")
        print("-" * 50)

        # Train
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, CONFIG['device']
        )

        # Validate
        val_loss, val_acc, cm = validate(
            model, val_loader, criterion, CONFIG['device']
        )

        # Update learning rate
        scheduler.step()
        current_lr = optimizer.param_groups[0]['lr']

        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['lr'].append(current_lr)

        print(f"\nEpoch Summary:")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        print(f"Learning Rate: {current_lr:.6f}")

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_cm = cm
            torch.save({
                'epoch': epoch,
                'fold': fold,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_acc': val_acc,
                'val_loss': val_loss,
                'class_names': class_names
            }, best_model_path)
            print(f"✓ New best model saved! Val Acc: {val_acc:.2f}%")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"Patience: {patience_counter}/{CONFIG['patience']}")

        # Save last model
        torch.save({
            'epoch': epoch,
            'fold': fold,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'class_names': class_names
        }, last_model_path)

        # Early stopping
        if patience_counter >= CONFIG['patience']:
            print(f"\nEarly stopping triggered after {epoch + 1} epochs")
            break
        
    # Save fold results
    fold_results = {
        'fold': fold + 1,
        'best_val_acc': best_val_acc,
        'final_val_acc': val_acc,
        'final_train_acc': train_acc,
        'final_val_loss': val_loss,
        'final_train_loss': train_loss,
        'epochs_trained': epoch + 1
    }
    all_fold_results.append(fold_results)

    # Save fold history
    df_history = pd.DataFrame(history)
    df_history.to_csv(os.path.join(fold_save_dir, 'training_history.csv'), index=False)
    print(f"Training history saved to: {os.path.join(fold_save_dir, 'training_history.csv')}")

    print("\n" + "=" * 50)
    print(f"Fold {fold + 1} Training Completed!")
    print("=" * 50)
    print(f"Best validation accuracy: {best_val_acc:.2f}%")
    print(f"Best model saved at: {best_model_path}")

    # Plot Confusion Matrix
    best_cm = best_cm.T
    plt.figure(figsize=(10, 8))
    plt.imshow(best_cm, cmap="Blues")
    plt.title("Normalized Confusion Matrix")
    plt.colorbar()

    plt.xticks(np.arange(len(class_names)), class_names, rotation=90)
    plt.yticks(np.arange(len(class_names)), class_names)

    plt.xlabel("True")
    plt.ylabel("Predicted")

    thresh = best_cm.max() / 2
    for i in range(best_cm.shape[0]):
        for j in range(best_cm.shape[1]):
            if best_cm[i, j] > 0:
                plt.text(j, i, f"{best_cm[i, j]}",
                        ha="center", va="center",
                        color="white" if best_cm[i, j] > thresh else "black")

    plt.savefig(os.path.join(fold_save_dir, 'confusion_matrix.png'), dpi=300, bbox_inches='tight')

    cm_norm = best_cm.astype("float") / best_cm.sum(axis=0, keepdims=True)

    plt.figure(figsize=(10, 8))
    plt.imshow(cm_norm, cmap="Blues")
    plt.title("Normalized Confusion Matrix")
    plt.colorbar()

    plt.xticks(np.arange(len(class_names)), class_names, rotation=90)
    plt.yticks(np.arange(len(class_names)), class_names)

    plt.xlabel("True")
    plt.ylabel("Predicted")

    thresh = cm_norm.max() / 2
    for i in range(cm_norm.shape[0]):
        for j in range(cm_norm.shape[1]):
            if cm_norm[i, j] > 0.009:
                plt.text(j, i, f"{cm_norm[i, j]:.2f}",
                        ha="center", va="center",
                        color="white" if cm_norm[i, j] > thresh else "black")
                
    plt.savefig(os.path.join(fold_save_dir, 'confusion_matrix_normalized.png'), dpi=300, bbox_inches='tight')

    # Plot training curves for this fold
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Plot loss
    axes[0].plot(history['train_loss'], label='Train Loss', marker='o')
    axes[0].plot(history['val_loss'], label='Val Loss', marker='s')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title(f'Fold {fold + 1} - Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True)

    # Plot accuracy
    axes[1].plot(history['train_acc'], label='Train Acc', marker='o')
    axes[1].plot(history['val_acc'], label='Val Acc', marker='s')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title(f'Fold {fold + 1} - Training and Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True)

    # Plot learning rate
    axes[2].plot(history['lr'], label='Learning Rate', marker='o', color='green')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Learning Rate')
    axes[2].set_title(f'Fold {fold + 1} - Learning Rate Schedule')
    axes[2].set_yscale('log')
    axes[2].legend()
    axes[2].grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(fold_save_dir, 'training_plots.png'), dpi=300, bbox_inches='tight')

    print(f"Training plots saved to: {os.path.join(fold_save_dir, 'training_plots.png')}")

# FIM DO LOOP DE FOLDS

In [ ]:
# Consolidate results from all folds
results_df = pd.DataFrame(all_fold_results)

print("\n" + "=" * 70)
print("CROSS-VALIDATION RESULTS SUMMARY")
print("=" * 70)
print(results_df.to_string(index=False))

print(f"\n{'=' * 70}")
print("STATISTICS")
print("=" * 70)
print(f"Mean Best Val Accuracy: {results_df['best_val_acc'].mean():.2f}% ± {results_df['best_val_acc'].std():.2f}%")
print(f"Mean Final Val Accuracy: {results_df['final_val_acc'].mean():.2f}% ± {results_df['final_val_acc'].std():.2f}%")
print(
    f"Mean Final Train Accuracy: {results_df['final_train_acc'].mean():.2f}% ± {results_df['final_train_acc'].std():.2f}%")
print(f"Mean Epochs Trained: {results_df['epochs_trained'].mean():.1f} ± {results_df['epochs_trained'].std():.1f}")

with open(os.path.join(CONFIG['save_dir'], 'cross_validation_stats.txt'), 'w') as f:
    f.write(f"Mean Best Val Accuracy: {results_df['best_val_acc'].mean():.2f}% ± {results_df['best_val_acc'].std():.2f}%\n")
    f.write(f"Mean Final Val Accuracy: {results_df['final_val_acc'].mean():.2f}% ± {results_df['final_val_acc'].std():.2f}%\n")
    f.write(f"Mean Final Train Accuracy: {results_df['final_train_acc'].mean():.2f}% ± {results_df['final_train_acc'].std():.2f}%\n")
    f.write(f"Mean Final Val Loss: {results_df['final_val_loss'].mean():.2f}% ± {results_df['final_val_loss'].std():.2f}%\n")
    f.write(f"Mean Final Train Loss: {results_df['final_train_loss'].mean():.2f}% ± {results_df['final_train_loss'].std():.2f}%\n")

# Save consolidated results
results_df.to_csv(os.path.join(CONFIG['save_dir'], 'cross_validation_results.csv'), index=False)
print(f"\nResults saved to: {os.path.join(CONFIG['save_dir'], 'cross_validation_results.csv')}")

# Plot comparison across folds
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Best validation accuracy per fold
axes[0].bar(results_df['fold'], results_df['best_val_acc'])
axes[0].axhline(y=results_df['best_val_acc'].mean(), color='r', linestyle='--', label='Mean')
axes[0].set_xlabel('Fold')
axes[0].set_ylabel('Best Validation Accuracy (%)')
axes[0].set_title('Best Validation Accuracy per Fold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Epochs trained per fold
axes[1].bar(results_df['fold'], results_df['epochs_trained'])
axes[1].axhline(y=results_df['epochs_trained'].mean(), color='r', linestyle='--', label='Mean')
axes[1].set_xlabel('Fold')
axes[1].set_ylabel('Epochs Trained')
axes[1].set_title('Epochs Trained per Fold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(CONFIG['save_dir'], 'cross_validation_comparison.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"Comparison plots saved to: {os.path.join(CONFIG['save_dir'], 'cross_validation_comparison.png')}")